In [2]:
%load_ext autoreload
%autoreload 2 

* 이미지를 VAE로 Embedding하여 얻은 vector의 similarity와 풍속데이터의 Similarity 비교
* 체크해야 할 것
    - 풍속기록이 있는 데이터만 필요하며, 일부는 Train 일부는 test 이를 구분해야 할 필요가 있음

In [65]:
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from collections import OrderedDict

import yaml
import torch
from torchvision import transforms, datasets
from torch.utils.data import SubsetRandomSampler, DataLoader, TensorDataset

from models import *
from src import *

In [8]:
# Wind data load

with open('data/_Main_dataset.pickle', 'rb') as f:
    windspeed_dataset = pickle.load(f)
    
station_list = list(windspeed_dataset.keys())

In [61]:
checkpoint['state_dict'].keys()

odict_keys(['model.encoder.0.0.weight', 'model.encoder.0.0.bias', 'model.encoder.1.0.weight', 'model.encoder.1.0.bias', 'model.encoder.2.0.weight', 'model.encoder.2.0.bias', 'model.encoder.3.0.weight', 'model.encoder.3.0.bias', 'model.fc.weight', 'model.fc.bias', 'model.fc_mu.weight', 'model.fc_mu.bias', 'model.fc_var.weight', 'model.fc_var.bias', 'model.decoder_input.weight', 'model.decoder_input.bias', 'model.decoder.0.0.weight', 'model.decoder.0.0.bias', 'model.decoder.1.0.weight', 'model.decoder.1.0.bias', 'model.decoder.2.0.weight', 'model.decoder.2.0.bias', 'model.final_layer.0.weight', 'model.final_layer.0.bias', 'model.final_layer.2.weight', 'model.final_layer.2.bias'])

In [78]:
list(checkpoint['state_dict'].values())[-2].size()

torch.Size([3, 32, 3, 3])

In [64]:
for name, param in model.state_dict().items():
    print(name)

encoder.0.0.weight
encoder.0.0.bias
encoder.1.0.weight
encoder.1.0.bias
encoder.2.0.weight
encoder.2.0.bias
encoder.3.0.weight
encoder.3.0.bias
fc.weight
fc.bias
fc_mu.weight
fc_mu.bias
fc_var.weight
fc_var.bias
decoder_input.weight
decoder_input.bias
decoder.0.0.weight
decoder.0.0.bias
decoder.1.0.weight
decoder.1.0.bias
decoder.2.0.weight
decoder.2.0.bias
final_layer.0.weight
final_layer.0.bias
final_layer.2.weight
final_layer.2.bias


In [69]:
# load config
config_path = 'configs/betatc_vae_sate.yaml'
checkpoint_path = 'logs/BetaTCVAE/_ckpt_epoch_83.ckpt'

with open(config_path, 'rb') as f:
    config = yaml.safe_load(f)

#load vae model
model = vae_models[config['model_params']['name']](**config['model_params'])

#checkpoint load
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu')) # when using CPU
new_state_dict = OrderedDict()
for k, v in checkpoint['state_dict'].items():
    name = k[6:]
    new_state_dict[name] = v
model.load_state_dict(new_state_dict)

RuntimeError: Error(s) in loading state_dict for BetaTCVAE:
	size mismatch for final_layer.2.weight: copying a param with shape torch.Size([3, 32, 3, 3]) from checkpoint, the shape in current model is torch.Size([1, 32, 3, 3]).
	size mismatch for final_layer.2.bias: copying a param with shape torch.Size([3]) from checkpoint, the shape in current model is torch.Size([1]).

In [56]:
# load img data

IMG_PATH = '../../Share_Data/satellite/jpg_hill_d10/jpg_tif_d10/{}az0.jpg'

img_list =[ ]
label_list = []
for station_code in station_list:
    img = Image.open(IMG_PATH.format(station_code)).resize([config['exp_params']['img_size']]*2)
    img_list.append(np.array(img))
    label_list.append(station_code)

tensor_img = TensorDataset(torch.Tensor(img_list))
dataloader = DataLoader(tensor_img)

In [6]:
windspeed_dataset

{'116': [37.4453,
  126.964,
      year  wind_speed
  0   1999    8.270264
  1   2000    6.839098
  2   2001    7.325929
  3   2002   11.660485
  4   2003    7.472565
  5   2004    9.719026
  6   2005    8.358245
  7   2006   15.132823
  8   2007   14.657723
  9   2008   13.021266
  10  2009   12.663475
  11  2010    9.322693
  12  2011    6.878651
  13  2012   13.851531
  14  2013    7.750261
  15  2014    7.944607
  16  2015    7.726704
  17  2016    6.607745
  18  2017    7.991721
  19  2018    7.332124
  20  2019    7.485245
  21  2020    8.215512,
  18.014452305569414],
 '160': [35.1188,
  129.0,
      year  wind_speed
  0   1999   11.129022
  1   2000   16.017739
  2   2001   11.824945
  3   2002   16.380080
  4   2003   21.119260
  5   2004   19.445593
  6   2005   13.475606
  7   2006   18.335567
  8   2007   14.441847
  9   2008   15.005487
  10  2009   14.303813
  11  2010   15.442596
  12  2011   28.981468
  13  2012   15.655399
  14  2013   12.492111
  15  2014   11.209542


In [ ]:
?

In [ ]:
# configs

config_path = 'configs/betatc_vae_sate.yaml'
with open(config_path) as f:
    config = yaml.safe_load(f)
    
config